In [18]:
from tensorflow import keras
import numpy as np
import librosa
import os
import soundfile as sf
from scipy.io import wavfile
from scipy.signal import butter, filtfilt, resample,lfilter
import numpy as np
import matplotlib.pyplot as plt
import librosa
import scipy.signal as signal
import librosa.display
import cv2
from PIL import Image

In [19]:
# Step 1: Denoising using a low pass filter
def apply_low_pass_filter(audio, sampling_rate, cutoff_freq):
    nyquist_freq = 0.5 * sampling_rate
    normalized_cutoff_freq = cutoff_freq / nyquist_freq
    b, a = signal.butter(4, normalized_cutoff_freq, btype='low', analog=False)
    denoised_audio = signal.lfilter(b, a, audio)
    return denoised_audio

In [20]:
# Downsampling audio
def downsample_audio(audio,original_sampling_rate,target_sampling_rate):
    resampled_audio = librosa.resample(audio, orig_sr=original_sampling_rate, target_sr=target_sampling_rate)
    return resampled_audio

In [21]:
# Split audio into fixed-length segments
def split_audio(audio, segment_length):
    num_segments = len(audio) // segment_length
    segments = [audio[i*segment_length:(i+1)*segment_length] for i in range(num_segments)]
    return segments

In [22]:
def save_spect_testing(data, sr, filename,i):
    # Extract spectrogram
    spectrogram = librosa.feature.melspectrogram(y=data, sr=sr)

    # Convert to decibels
    spectrogram_db = librosa.power_to_db(spectrogram, ref=np.max)

    # Plot spectrogram
    plt.figure(figsize=(1.28,1.28))
    librosa.display.specshow(spectrogram_db, sr=sr)
    plt.savefig(("/Users/vaishnavikrovvidi/Desktop/Dataset_A/Test-A/{}_{}.png").format(filename,i), transparent=True)
    plt.close()

In [29]:
def pred(audio_path, model_path,filename):
    audio, sampling_rate = librosa.load(audio_path, sr=None)
    # Denoising
    cutoff_frequency = 195
    denoised_audio = apply_low_pass_filter(audio, sampling_rate, cutoff_frequency)

    # Downsampling
    target_sampling_rate = sampling_rate // 10
    downsampled_audio = downsample_audio(denoised_audio, sampling_rate, target_sampling_rate)

    # Splitting audio
    segment_length = target_sampling_rate * 3
    segments = split_audio(downsampled_audio, segment_length)
    model = keras.models.load_model(model_path)
    i=0
    
    for segment in segments:
        save_spect_testing(segment,target_sampling_rate,filename,i)

        img=Image.open(("/Users/vaishnavikrovvidi/Desktop/Dataset_A/Test-A/{}_{}.png").format(filename,i)).convert('RGB')
        img_arr=np.asarray(img)
        img_arr=img_arr/255

        img_arr = img_arr.reshape(1, 128, 128, 3)
        
        prediction = model.predict(img_arr)
        x=np.argmax(prediction)
        confidence = prediction[0, x]
        i=i+1

#         prediction = model.predict(img_arr)
#         x=np.argmax(prediction)

        classes={0:'Artifact', 1:'Extrasystole', 2:'Murmur', 3:'Normal'}
        print(classes[x],confidence)


In [30]:
audio_path='/Users/vaishnavikrovvidi/Desktop/Extrasystole.wav'
model_path='/Users/vaishnavikrovvidi/Heartdisease/heart_resnet_spectro.hdf5'
filename='fgq'
pred(audio_path, model_path, filename)

2023-06-30 11:56:33.571709: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


1/1 [==============================] - 1s 1s/step
Extrasystole 0.9407989
1/1 [==============================] - 0s 61ms/step
Extrasystole 0.9466342
